** Skeleton of Assignment 4:**
    test if the distribution of 
    
    1) trip duration of bikers that ride during the day vs night
    
    2) age of bikers for trips originating in Manhattan and in Brooklyn
    
    are different. Use 3 tests: KS, Pearson's, Spearman's. 
    
    Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 
    
    For the KS do the test with the entire dataset and with a subset 200 times smaller
    
    Choose a single significant threshold for the whole exercise. 
    
    For each test phrase the Null Hypothesis in words.
    
    Describe the return of the scipy function you use in each case.
    
    State the result in terms of rejection of the Null.

In [7]:
# my usual imports and setups
from __future__ import print_function, division
import pylab as pl
import pandas as pd
import numpy as np

import scipy.stats
import os
#this makes my plots pretty! but it is totally not mandatory to do it
import json

if os.getenv ('PUI2016') is None:
    print ("Must set env variable PUI2016")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUI2018")
    
import os
import json

#set this to true while developing to subsample data
DEVELOPING = False

#chose day hours vs night hours:
DAYTIME = (6, 18)

In [8]:
os.getenv('PUIDATA')

'/nfshome/hh2264/PUIdata'

** Statistical significance threshold throughout: $\alpha = 0.05$ **

In [2]:
alpha = 0.05

# Read in data
I am reading in data from August 2014 and Feburary 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [19]:
datestring08 = '201408'
datestring02 = '201502'

In [20]:
# read in data August
cb201408 = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring08 + '-citibike-tripdata.csv')
cb201408.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1142,2014-08-01 00:00:04,2014-08-01 00:19:06,470,W 20 St & 8 Ave,40.743453,-74.000040,312,Allen St & E Houston St,40.722055,-73.989111,19117,Subscriber,1969,1
1,117,2014-08-01 00:00:05,2014-08-01 00:02:02,236,St Marks Pl & 2 Ave,40.728419,-73.987140,432,E 7 St & Avenue A,40.726218,-73.983799,20549,Subscriber,1991,1
2,546,2014-08-01 00:00:07,2014-08-01 00:09:13,224,Spruce St & Nassau St,40.711464,-74.005524,340,Madison St & Clinton St,40.712690,-73.987763,15997,Subscriber,1984,1
3,2126,2014-08-01 00:00:11,2014-08-01 00:35:37,150,E 2 St & Avenue C,40.720874,-73.980858,522,E 51 St & Lexington Ave,40.757148,-73.972078,21437,Subscriber,1988,1
4,329,2014-08-01 00:00:15,2014-08-01 00:05:44,519,E 42 St & Vanderbilt Ave,40.752416,-73.978370,477,W 41 St & 8 Ave,40.756405,-73.990026,16693,Subscriber,1981,1


In [21]:
#read in data Feburary
cb201502 = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring02 + '-citibike-tripdata.csv')
cb201502.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,801,2/1/2015 0:00,2/1/2015 0:14,521,8 Ave & W 31 St,40.750450,-73.994811,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978.0,2
1,379,2/1/2015 0:00,2/1/2015 0:07,497,E 17 St & Broadway,40.737050,-73.990093,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993.0,1
2,2474,2/1/2015 0:01,2/1/2015 0:42,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969.0,2
3,818,2/1/2015 0:01,2/1/2015 0:15,2004,6 Ave & Broome St,40.724399,-74.004704,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985.0,2
4,544,2/1/2015 0:01,2/1/2015 0:10,323,Lawrence St & Willoughby St,40.692362,-73.986317,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957.0,1


In [24]:
#subsample data while developing code 
#THIS IS IMPORTANT! the computing speed when using the full sample is orohibitively slow
#for a good workflow it was necessary to develope on a subsample, then run the whole notebook on the full dataset

if DEVELOPING:
    cb2015 = cb201408[::1000]
else:
    cb2015 = pd.concat([cb201408, cb201502])

In [25]:
print ("Shape of CB DF")
cb2015.shape

Shape of CB DF


(1160419, 15)

In [26]:
cb2015['date'] = pd.to_datetime(cb2015.starttime)
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 
cb2015.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,gender
count,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06,1.160419e+06
mean,8.445177e+02,4.511436e+02,4.073454e+01,-7.399107e+01,4.522944e+02,4.073425e+01,-7.399124e+01,1.812846e+04,1.078735e+00
std,8.956909e+02,3.710098e+02,1.938657e-02,1.226268e-02,3.748055e+02,1.941668e-02,1.230033e-02,2.083937e+03,5.577636e-01
min,6.000000e+01,7.200000e+01,4.068034e+01,-7.401713e+01,7.200000e+01,4.068034e+01,-7.401713e+01,1.452900e+04,0.000000e+00
25%,3.970000e+02,3.050000e+02,4.072110e+01,-7.399998e+01,3.050000e+02,4.072083e+01,-7.400026e+01,1.633600e+04,1.000000e+00
50%,6.330000e+02,4.040000e+02,4.073625e+01,-7.399076e+01,4.040000e+02,4.073588e+01,-7.399093e+01,1.811000e+04,1.000000e+00
75%,1.048000e+03,4.890000e+02,4.075007e+01,-7.398195e+01,4.890000e+02,4.074916e+01,-7.398205e+01,1.989800e+04,1.000000e+00
max,4.301600e+04,3.002000e+03,4.077152e+01,-7.395005e+01,3.002000e+03,4.077152e+01,-7.395005e+01,2.170300e+04,2.000000e+00


In [27]:
cb2015.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'date'],
      dtype='object')

## SPLIT BY CATEGORY

splitting data by day vs night and looking at trip duration:

**H0: there is no statistical difference in the trip duration distribution during the day and the night**
$$ \alpha = 0.05 $$

extracting the age happens in the cell below

In [28]:
#extract hour when trip started
cb2015['starthour'] = pd.to_numeric(cb2015.starttime.apply(lambda x: 
                                                           x.split()[1].split(':')[0]))

In [29]:
#check start hour
cb2015.starthour.values[0]

0

In [30]:
# split day and night smaples
# chose trip starting between 6 and 18 as day trips
cb2015['dayride'] = (cb2015.starthour.values > DAYTIME[0]) & (cb2015.starthour.values < DAYTIME[1])

In [31]:
#collecting latitude and longitude for later splitting borroughs
latlon = (cb2015[['start station latitude', 
                'start station longitude']].values.astype(float))
print("The number of rows in the 2015 citibike dataset is", len(latlon))
#dropping not needed columns to lighten memory load
cb2015 = cb2015.drop(cb2015.columns[5:-1], axis=1).drop(cb2015.columns[1:3], axis=1)
cb2015.head()

The number of rows in the 2015 citibike dataset is 1160419


,tripduration,start station id,start station name,dayride
0,1142,470,W 20 St & 8 Ave,False
1,117,236,St Marks Pl & 2 Ave,False
2,546,224,Spruce St & Nassau St,False
3,2126,150,E 2 St & Avenue C,False
4,329,519,E 42 St & Vanderbilt Ave,False


In [32]:
#your df should look something like this
# look at these data carefully... you may see someinteresting values!
cb2015.describe()

,tripduration,start station id
count,1.160419e+06,1.160419e+06
mean,8.445177e+02,4.511436e+02
std,8.956909e+02,3.710098e+02
min,6.000000e+01,7.200000e+01
25%,3.970000e+02,3.050000e+02
50%,6.330000e+02,4.040000e+02
75%,1.048000e+03,4.890000e+02
max,4.301600e+04,3.002000e+03


# exploratory analysis

In [33]:
print ("Number of daytime rides", cb2015.dayride.sum())
print ("Number of nighttime rides", (~cb2015.dayride).sum())

Number of daytime rides 770185
Number of nighttime rides 390234


In [35]:
# plot daytime trip duration histogram in 100 bins
# plot 0% to 95% percentile because I expect these distribution to be long tailed and I see 43,000 hour trips above
pc = np.percentile(cb2015.tripduration, [0, 99])
bins = np.linspace(int(pc[0]), int(pc[1] / 100) * 100, 100)
dayax = cb2015[cb2015.dayride].tripduration.groupby(
    pd.cut(cb2015[cb2015.dayride].tripduration, bins)).\
                    agg([count_nonzero]).plot(kind='bar', legend=False)
dayax.set_title("August 2014 and Feburary 2015 rides")
# plot nighttime trip duration histogram
cb2015[~cb2015.dayride].tripduration.groupby(
    pd.cut(cb2015[~cb2015.dayride].tripduration, bins)).\
                    agg([count_nonzero]).plot(kind='bar', legend=False, ax=dayax, 
                                             color='IndianRed')
xlab = dayax.get_xticklabels()
dayax.set_xticklabels(['' if float(i)%10 > 0 else xlab[i] for i in range(len(xlab))]);
dayax.set_xlabel(" trip duration (minutes)");



NameError: name 'count_nonzero' is not defined

In [36]:
print ("Figure 1: histogrammed distribution of ride duration by day (blue) and night (red).\n" +
       "The distributions are plotted up to the 95th percentile (removing outliers in the tail).\n" +
       "The distributions look similar, such that visual inspection is not sufficient " +
       "to determine if there is a difference." + 
       "The number of rides is obviously higher during day hours.\n" +
       "Note, that our definition of day includes %d hours between hour %d and %d"%(DAYTIME[1] - DAYTIME[0],
                                                                                    DAYTIME[0], DAYTIME[1]))

Figure 1: histogrammed distribution of ride duration by day (blue) and night (red).
The distributions are plotted up to the 95th percentile (removing outliers in the tail).
The distributions look similar, such that visual inspection is not sufficient to determine if there is a difference.The number of rides is obviously higher during day hours.
Note, that our definition of day includes 12 hours between hour 6 and 18


In [37]:
#print df.ageS, df.ageS.cumsum()

csD = cb2015[cb2015.dayride].tripduration.groupby(pd.cut(cb2015[cb2015.dayride].tripduration, 
                                                 bins)).agg([count_nonzero]).cumsum()

csN = cb2015[~cb2015.dayride].tripduration.groupby(pd.cut(cb2015[~cb2015.dayride].tripduration, 
                                                  bins)).agg([count_nonzero]).cumsum()

#print (np.abs(csD / csD.max()-csN / csN.max()))

pl.plot(bins[:-1] + 5, csD / csD.max(), label = "day")
pl.plot(bins[:-1] + 5, csN / csN.max(), label = "night")
pl.plot(bins[:-1] + 5, np.abs(csN / csN.max() - csD / csD.max()), 'k-',
        label = "difference")
pl.xlabel("trip duration (minutes)")
pl.ylabel("Normalized Cumulative Number")
pl.legend();

NameError: name 'count_nonzero' is not defined

** Figure 2: the cumulative distribution of CitiBike riders' ages by day vs night trips.** 

The cumulative distribution is, by eye, practically identical with the maximum difference in heights between the cumulative distributions reached near 100 minutes of travel time. The difference is much less than 10%.

# KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

Null: the two distribution come from the same parent distribution

In [38]:
ks = scipy.stats.ks_2samp(cb2015[cb2015.dayride].tripduration, cb2015[~cb2015.dayride].tripduration)
print (ks)
if ks[1] >= alpha: 
    print ("The Null hypothesis that the two samples are generated " + 
            "by the same parent distribution cannot be rejected (p>=0.05)")
else: 
    print ("The Null hypothesis that the two samples are generated " + 
            "by the same parent distribution is rejected (p<0.05)")
    

Ks_2sampResult(statistic=0.012943463085704487, pvalue=3.9237493155841106e-38)


NameError: name 'alpha' is not defined

** Note on the KS test: the KS test, and generally any test of Null Hypothesis Statistical Testing, has higher power for larger samples. With a million observations, 100s of thousands in each sample tiny differences tend to become statistically significant. ** (The distribution looked different by eye in a cumulative plot in the testig phase, with 1000 fewer samples but the p-value was less significant)

** Now retest using a test for correlation. **

The null hypothesis is that the two samples are generated from the same distribution. 

The alternative hypothesis is that the two samples are generated by unrelated distributions.

Note that this is not a rigorous test, unlike the KS test, and according to scipy.stats.pearsonr _"The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so."_


In [39]:
#more rides during the day than during the night: subsample day rides
dfday = cb2015[cb2015.dayride].iloc[np.random.choice(range(len(cb2015[cb2015.dayride])), (~cb2015.dayride).sum(), 
                                      replace=False)]

In [40]:
parsons = scipy.stats.pearsonr(np.sort(dfday.tripduration), np.sort(cb2015[~cb2015.dayride].tripduration))
print ("Parson's test result", parsons)
if parsons[1] >= alpha: 
    print (r"The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "cannot be rejected (p>=0.05)")
else: 
    print ("The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "is rejected (p = %.3f<0.05)"%parsons[1])
 

Parson's test result (0.97355013945918534, 0.0)


NameError: name 'alpha' is not defined

# Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

In [41]:
spearmanr = scipy.stats.spearmanr(np.sort(dfday.tripduration), np.sort(cb2015[~cb2015.dayride].tripduration))
print ("Spearman's r test result", spearmanr)
if spearmanr[1] >= alpha: 
    print ("The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "cannot be rejected (p>=0.05)")
else: 
    print ("The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "is rejected (p = %.3f<0.05)"%spearmanr[1])
 

Spearman's r test result SpearmanrResult(correlation=0.99999931635544259, pvalue=0.0)


NameError: name 'alpha' is not defined

** This test is often used to assess if sample are generated by the same distribution but in fact it tests if the samples are generated by _correlated_ distributions. As a test to assess independence it is not powerful, as even a small amount of correlation would raise the p-value**